In [1]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("spam.csv", encoding="ISO-8859-1")

In [3]:
df = df.rename(columns={'v1': 'label', 'v2': 'text'})
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], errors='ignore')
df = df.dropna(subset=['label', 'text'])

In [4]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [5]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   int64 
 1   text    5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.shape

(5572, 2)

In [7]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [8]:
df['label'].value_counts()

label
0    4825
1     747
Name: count, dtype: int64

# Preparing for NLtk Preprocessing

In [9]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Hello\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hello\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# NLP

In [18]:
def preprocess_data(input_text):
    preprocessed_data = []
    for text in input_text:
        lowered = text.lower()
        translator = str.maketrans('', '', string.punctuation)
        cleaned_text = lowered.translate(translator)
        tokenized = nltk.word_tokenize(cleaned_text)
        stop_words_removed = [word for word in tokenized if word not in stop_words]
        stemmed = [stemmer.stem(word) for word in stop_words_removed]
        preprocessed_data.append(' '.join(stemmed))
    return preprocessed_data

# Preprocessing the text data

In [19]:
# Preprocess text data
preprocessed_data = preprocess_data(df['text'])


In [20]:
# Convert text to numerical features
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(preprocessed_data)
y = df['label']


In [21]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_vec, y)

In [23]:
y_resampled.value_counts()

label
0    4825
1    4825
Name: count, dtype: int64

# Splitting data for test and train

In [24]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

# Model Evaluation

In [25]:
# Make predictions
y_pred = model.predict(X_test)


In [26]:
# Evaluate model
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:", classification_report(y_test, y_pred))


Naive Bayes Accuracy: 0.9761658031088083
Classification Report:               precision    recall  f1-score   support

           0       0.97      0.98      0.98       985
           1       0.98      0.97      0.98       945

    accuracy                           0.98      1930
   macro avg       0.98      0.98      0.98      1930
weighted avg       0.98      0.98      0.98      1930



# New predictions

In [36]:
# Define new input text
new_text_input = np.array(["Hey, are we still meeting for coffee at 5 PM?"])

# Preprocess the new input text
new_preprocessed = preprocess_data(new_text_input)

# Convert text to numerical features using the trained vectorizer
user_input_vec = vectorizer.transform(new_preprocessed)

# Predict using the trained Naive Bayes model
prediction = model.predict(user_input_vec)

In [37]:
# Display the result
print("Spam" if prediction[0] == 1 else "Not Spam")  # 1 = Spam, 0 = Not Spam

Not Spam


# Saving the model

In [31]:
import pickle

In [32]:
with open('model.pkl', 'wb') as file:
  pickle.dump(model, file)

In [33]:
with open('vectorizer.pkl', 'wb') as file:
  pickle.dump(vectorizer, file)

# Loading the model

In [34]:
with open('model.pkl', 'rb') as file:
  loaded_model = pickle.load(file)

In [35]:
loaded_model.predict(user_input_vec)

array([1])

 # Sample inputs

In [ ]:
Spam Messages
"Congratulations! You've won a $1,000 Walmart gift card. Go to www.fakeurl.com to claim now!"
"URGENT! Your account has been compromised. Reply with your PIN to secure your account."
"FREE entry to our $10,000 prize draw! Text WIN to 12345 now!"
"You’ve been selected for a guaranteed loan! Apply now: www.loanscam.com."
"Act now! Get 50% off on your next purchase. Limited time only!"


In [ ]:
Non-Spam (Ham) Messages
"Hey, are we still meeting for coffee at 5 PM?"
"Don’t forget to bring your laptop for the meeting tomorrow."
"Happy birthday! Have an amazing day filled with love and laughter!"
"Call me when you get a chance. Need to discuss something important."
"I’m running late. Will be there in 10 minutes."
